In [1]:
import plotly.graph_objects as go

from ta.trend import MACD
from ta.momentum import StochasticOscillator

import numpy as np
import pandas as pd
from pykrx import stock
from pykrx import bond
from time import sleep

from datetime import datetime
from datetime import timedelta
import os
import time
from plotly.subplots import make_subplots
import glob

### 데이터 로드

In [2]:
ohlcv_df_raw = pd.read_csv('ohlcv_df_raw.csv', dtype = {'ticker':str})
ticker_list = ohlcv_df_raw['ticker'].unique()

In [4]:
df_raw_total = pd.DataFrame()
df_raw_anal_total = pd.DataFrame()

for ticker_nm in ticker_list:
    df_raw = ohlcv_df_raw[ohlcv_df_raw['ticker'] == ticker_nm].reset_index(drop = True)
    
    ######################################################################
    # 보조지표
    ###################################################################### 

    # 이동평균선
    df_raw['MA5'] = df_raw['close'].rolling(window=5).mean()
    df_raw['MA20'] = df_raw['close'].rolling(window=20).mean()
    df_raw['MA60'] = df_raw['close'].rolling(window=60).mean()
    df_raw['MA120'] = df_raw['close'].rolling(window=120).mean()

    # 볼린저밴드
    std = df_raw['close'].rolling(20).std(ddof=0)

    df_raw['upper'] = df_raw['MA20'] + 2 * std
    df_raw['lower'] = df_raw['MA20'] - 2 * std

    # MACD
    # MACD 
    macd = MACD(close=df_raw['close'], 
                window_slow=26,
                window_fast=12, 
                window_sign=9)


    df_raw['MACD_DIFF'] = macd.macd_diff()
    df_raw['MACD'] = macd.macd()
    df_raw['MACD_Signal'] = macd.macd_signal()

    # RSI
    df_raw['변화량'] = df_raw['close'] - df_raw['close'].shift(1)
    df_raw['상승폭'] = np.where(df_raw['변화량']>=0, df_raw['변화량'], 0)
    df_raw['하락폭'] = np.where(df_raw['변화량'] <0, df_raw['변화량'].abs(), 0)

    # welles moving average
    df_raw['AU'] = df_raw['상승폭'].ewm(alpha=1/14, min_periods=14).mean()
    df_raw['AD'] = df_raw['하락폭'].ewm(alpha=1/14, min_periods=14).mean()
    df_raw['RSI'] = df_raw['AU'] / (df_raw['AU'] + df_raw['AD']) * 100

    df_raw['MA5-20'] = df_raw['MA5'] - df_raw['MA20']
    df_raw['MA20-60'] = df_raw['MA20'] - df_raw['MA60']
    df_raw['MA60-120'] = df_raw['MA60'] - df_raw['MA120']
    
    
    ######################################################################
    # 보조지표 분석
    ###################################################################### 
    df_raw_anal = df_raw[['date','ticker', 'close']]
    
    # 골든크로스
    # 골든 크로스 5-20
    # 음수에서 양수로 바뀌는 모든 인덱스 찾기
    idx_5_20_gold_cross = [idx for idx in range(len(df_raw)) if df_raw["MA5-20"].iloc[idx] > 0 and df_raw["MA5-20"].iloc[idx - 1] <= 0]

    # 데드 크로스 5-20
    # 양수에서 음수로 바뀌는 모든 인덱스 찾기
    idx_5_20_dead_cross = [idx for idx in range(len(df_raw)) if df_raw["MA5-20"].iloc[idx] < 0 and df_raw["MA5-20"].iloc[idx - 1] >= 0]

    # 골든 크로스 20-60
    # 음수에서 양수로 바뀌는 모든 인덱스 찾기
    idx_20_60_gold_cross = [idx for idx in range(len(df_raw)) if df_raw["MA20-60"].iloc[idx] > 0 and df_raw["MA20-60"].iloc[idx - 1] <= 0]

    # 골든 크로스 20-60
    # 음수에서 양수로 바뀌는 모든 인덱스 찾기
    idx_20_60_dead_cross = [idx for idx in range(len(df_raw)) if df_raw["MA20-60"].iloc[idx] < 0 and df_raw["MA20-60"].iloc[idx - 1] >= 0]
    

    df_raw_anal.loc[:, '5_20_cross'] = '-'
    df_raw_anal.loc[idx_5_20_gold_cross,'5_20_cross'] = '골든크로스(매수)'
    df_raw_anal.loc[idx_5_20_dead_cross,'5_20_cross'] = '데드크로스(매도)'

    df_raw_anal.loc[:, '20_60_cross'] = '-'
    df_raw_anal.loc[idx_20_60_gold_cross,'20_60_cross'] = '골든크로스(매수)'
    df_raw_anal.loc[idx_20_60_dead_cross,'20_60_cross'] = '데드크로스(매도)'
    
    
    # 정배열 역배열
    ascending_sq  = (df_raw['MA5-20'] > 0) & \
    (df_raw['MA20-60'] > 0) & \
    (df_raw['MA60-120'] > 0) 

    descending_sq  = (df_raw['MA5-20'] < 0) & \
    (df_raw['MA20-60'] < 0) & \
    (df_raw['MA60-120'] < 0) 
    
    df_raw_anal.loc[:,'array'] = '-'
    df_raw_anal.loc[ascending_sq,'array'] = '정배열(매수)'
    df_raw_anal.loc[descending_sq,'array'] = '역배열(매도)'
    
    
    # 볼린저밴드
    down_reg_sq = df_raw['upper'] - df_raw['close'] 
    top_reg_sq  = df_raw['lower'] - df_raw['close'] 

    down_reg = [idx for idx in range(1,len(df_raw)) if down_reg_sq[idx] > 0 and down_reg_sq[idx-1] <= 0]
    top_reg = [idx for idx in range(1,len(df_raw)) if top_reg_sq[idx] < 0 and top_reg_sq[idx-1] >= 0]
    
    df_raw_anal.loc[:,'Bollinger_band'] = '-'
    df_raw_anal.loc[ascending_sq,'Bollinger_band'] = '하향회귀(매도)'
    df_raw_anal.loc[descending_sq,'Bollinger_band'] = '상향회귀(매수)'
    
    
    # MACD
    signal_down_cross = [idx for idx in range(1,len(df_raw)) if df_raw['MACD_DIFF'][idx] < 0 and df_raw['MACD_DIFF'][idx-1] >= 0]
    signal_top_corss = [idx for idx in range(1,len(df_raw)) if df_raw['MACD_DIFF'][idx] > 0 and df_raw['MACD_DIFF'][idx-1] <= 0]

    df_raw_anal.loc[:,'MACD'] = '-'
    df_raw_anal.loc[signal_down_cross,'MACD'] = '하향돌파(매도)'
    df_raw_anal.loc[signal_top_corss,'MACD'] = '상향돌파(매수)'    
    
    # RSI
    down_reg = [idx for idx in range(1,len(df_raw)) if df_raw['RSI'][idx] > 70 and df_raw['RSI'][idx-1] <= 70]
    top_reg = [idx for idx in range(1,len(df_raw)) if df_raw['RSI'][idx] < 30 and df_raw['RSI'][idx-1] >= 30]
    
    
    df_raw_anal.loc[:,'RSI'] = '-'
    df_raw_anal.loc[down_reg,'RSI'] = 'RSI 상단 하향돌파(매도)'
    df_raw_anal.loc[top_reg,'RSI'] = 'RSI 하단 상향 돌파(매수)'


    df_raw_total = pd.concat([df_raw_total, df_raw])
    df_raw_anal_total = pd.concat([df_raw_anal_total, df_raw_anal])
    
    print(ticker_nm)

/tmp/ipykernel_1782644/984985259.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:, '5_20_cross'] = '-'
/tmp/ipykernel_1782644/984985259.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:, '20_60_cross'] = '-'
/tmp/ipykernel_1782644/984985259.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

005930
373220


/tmp/ipykernel_1782644/984985259.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:, '5_20_cross'] = '-'
/tmp/ipykernel_1782644/984985259.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:, '20_60_cross'] = '-'
/tmp/ipykernel_1782644/984985259.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

000660
207940


/tmp/ipykernel_1782644/984985259.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:, '5_20_cross'] = '-'
/tmp/ipykernel_1782644/984985259.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:, '20_60_cross'] = '-'
/tmp/ipykernel_1782644/984985259.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

...

/tmp/ipykernel_1782644/984985259.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:, '5_20_cross'] = '-'
/tmp/ipykernel_1782644/984985259.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:, '20_60_cross'] = '-'
/tmp/ipykernel_1782644/984985259.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

001450
004370


/tmp/ipykernel_1782644/984985259.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:,'MACD'] = '-'
/tmp/ipykernel_1782644/984985259.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:,'RSI'] = '-'
/tmp/ipykernel_1782644/984985259.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

071050
180640


/tmp/ipykernel_1782644/984985259.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:, '5_20_cross'] = '-'
/tmp/ipykernel_1782644/984985259.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_anal.loc[:, '20_60_cross'] = '-'
/tmp/ipykernel_1782644/984985259.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [5]:
df_raw_total.reset_index(drop = True)

,date,open,high,low,close,volume,price_change_percentage,ticker,corp_name,MA5,...,MACD_Signal,변화량,상승폭,하락폭,AU,AD,RSI,MA5-20,MA20-60,MA60-120
0,2018-01-02,51380,51400,50780,51020,169485,0.117739,005930,삼성전자,NaN,...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-03,52540,52560,51420,51620,200270,1.176009,005930,삼성전자,NaN,...,NaN,600.0,600.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-04,52120,52180,50640,51080,233909,-1.046106,005930,삼성전자,NaN,...,NaN,-540.0,0.0,540.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-01-05,51300,52120,51200,52120,189623,2.036022,005930,삼성전자,NaN,...,NaN,1040.0,1040.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-01-08,52400,52520,51500,52020,167673,-0.191865,005930,삼성전자,51572.0,...,NaN,-100.0,0.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130660,2023-09-26,41800,42200,40350,40600,119412,-5.140187,180640,한진칼,41000.0,...,-555.033178,-2200.0,0.0,2200.0,746.607928,918.450001,44.839757,-1692.5,-1292.500000,-452.916667
130661,2023-09-27,40200,42900,40200,42350,107323,4.310345,180640,한진칼,41230.0,...,-578.254931,1750.0,1750.0,0.0,818.278790,852.846429,48.965738,-1405.0,-1279.166667,-503.333333
130662,2023-10-04,41750,42150,40200,40250,94897,-4.958678,180640,한진칼,41250.0,...,-616.232780,-2100.0,0.0,2100.0,759.830305,941.928827,44.649698,-1175.0,-1389.166667,-573.750000
130663,2023-10-05,40450,41200,40300,41200,41211,2.360248,180640,한진칼,41440.0,...,-644.907388,950.0,950.0,0.0,773.413855,874.648197,46.928685,-912.5,-1375.000000,-638.750000


In [6]:
df_raw_anal_total

,date,ticker,close,5_20_cross,20_60_cross,array,Bollinger_band,MACD,RSI
0,2018-01-02,005930,51020,-,-,-,-,-,-
1,2018-01-03,005930,51620,-,-,-,-,-,-
2,2018-01-04,005930,51080,-,-,-,-,-,-
3,2018-01-05,005930,52120,-,-,-,-,-,-
4,2018-01-08,005930,52020,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...
1415,2023-09-26,180640,40600,-,-,역배열(매도),상향회귀(매수),-,-
1416,2023-09-27,180640,42350,-,-,역배열(매도),상향회귀(매수),-,-
1417,2023-10-04,180640,40250,-,-,역배열(매도),상향회귀(매수),-,-
1418,2023-10-05,180640,41200,-,-,역배열(매도),상향회귀(매수),-,-
